In [8]:
# importing the modules
import json 
import pandas as pd 
from pandas.io.json import json_normalize #package for flattening json in pandas df


In [4]:
data = [
    {"Roll no": 1,
     "student": {"first_name": "Ram", "last_name": "kumar"}
     },
    {"student": {"English": "95", "Math": "88"}
     },
    {"Roll no": 2,
     "student": {"first_name": "Joseph", "English": "90", "Science": "82"}
     },
    {"Roll no": 3,
     "student": {"first_name": "abinaya", "last_name": "devi"},
     "student": {"English": "91", "Math": "98"}
     },
]

print(data)

[{'Roll no': 1, 'student': {'first_name': 'Ram', 'last_name': 'kumar'}}, {'student': {'English': '95', 'Math': '88'}}, {'Roll no': 2, 'student': {'first_name': 'Joseph', 'English': '90', 'Science': '82'}}, {'Roll no': 3, 'student': {'English': '91', 'Math': '98'}}]


In [3]:
df = pd.json_normalize(data)
print(df)

   Roll no student.first_name student.last_name student.English student.Math  \
0      1.0                Ram             kumar             NaN          NaN   
1      NaN                NaN               NaN              95           88   
2      2.0             Joseph               NaN              90          NaN   
3      3.0                NaN               NaN              91           98   

  student.Science  
0             NaN  
1             NaN  
2              82  
3             NaN  


In [6]:
data = [
    {
        "company": "Google",
        "tagline": "Dont be evil",
        "management": {"CEO": "Sundar Pichai"},
        "department": [
            {"name": "Gmail", "revenue (bn)": 123},
            {"name": "GCP", "revenue (bn)": 400},
            {"name": "Google drive", "revenue (bn)": 600},
        ],
    },
    {
        "company": "Microsoft",
        "tagline": "Be What's Next",
        "management": {"CEO": "Satya Nadella"},
        "department": [
            {"name": "Onedrive", "revenue (bn)": 13},
            {"name": "Azure", "revenue (bn)": 300},
            {"name": "Microsoft 365", "revenue (bn)": 300},
        ],
    },
  
]
print(data)

[{'company': 'Google', 'tagline': 'Dont be evil', 'management': {'CEO': 'Sundar Pichai'}, 'department': [{'name': 'Gmail', 'revenue (bn)': 123}, {'name': 'GCP', 'revenue (bn)': 400}, {'name': 'Google drive', 'revenue (bn)': 600}]}, {'company': 'Microsoft', 'tagline': "Be What's Next", 'management': {'CEO': 'Satya Nadella'}, 'department': [{'name': 'Onedrive', 'revenue (bn)': 13}, {'name': 'Azure', 'revenue (bn)': 300}, {'name': 'Microsoft 365', 'revenue (bn)': 300}]}]


In [7]:
result = pd.json_normalize(
    data, "department", ["company", "tagline", ["management", "CEO"]]
)
result

,name,revenue (bn),company,tagline,management.CEO
0,Gmail,123,Google,Dont be evil,Sundar Pichai
1,GCP,400,Google,Dont be evil,Sundar Pichai
2,Google drive,600,Google,Dont be evil,Sundar Pichai
3,Onedrive,13,Microsoft,Be What's Next,Satya Nadella
4,Azure,300,Microsoft,Be What's Next,Satya Nadella
5,Microsoft 365,300,Microsoft,Be What's Next,Satya Nadella


In [10]:
#load json object
with open('datasets/raw_nyc_phil.json') as f:
    d = json.load(f)

In [12]:
nycphil = pd.json_normalize(d['programs'])
nycphil.head(3)

,season,orchestra,concerts,programID,works,id
0,1842-43,New York Philharmonic,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':...",3853,"[{'workTitle': 'SYMPHONY NO. 5 IN C MINOR, OP....",38e072a7-8fc9-4f9a-8eac-3957905c0002
1,1842-43,New York Philharmonic,"[{'Date': '1843-02-18T05:00:00Z', 'eventType':...",5178,[{'workTitle': 'SYMPHONY NO. 3 IN E FLAT MAJOR...,c7b2b95c-5e0b-431c-a340-5b37fc860b34
2,1842-43,Musicians from the New York Philharmonic,"[{'Date': '1843-04-07T05:00:00Z', 'eventType':...",10785,"[{'workTitle': 'EGMONT, OP.84', 'composerName'...",894e1a52-1ae5-4fa7-aec0-b99997555a37


In [22]:
nycphil.shape

(13954, 6)

In [24]:
nycphil.columns

Index(['season', 'orchestra', 'concerts', 'programID', 'works', 'id'], dtype='object')

In [18]:
nycphil['works']

0        [{'workTitle': 'SYMPHONY NO. 5 IN C MINOR, OP....
1        [{'workTitle': 'SYMPHONY NO. 3 IN E FLAT MAJOR...
2        [{'workTitle': 'EGMONT, OP.84', 'composerName'...
3        [{'workTitle': 'SYMPHONY NO. 2 IN D MAJOR, OP....
4        [{'workTitle': 'SYMPHONY NO. 7 IN A MAJOR, OP....
                               ...                        
13949    [{'workTitle': 'QUARTET, STRING, NO. 3 (MISHIM...
13950    [{'workTitle': 'SYMPHONY NO. 9, E MINOR, OP.95...
13951    [{'workTitle': 'SYMPHONY NO.  7 IN E MINOR (SO...
13952    [{'workTitle': 'SYMPHONY NO. 9, E MINOR, OP.95...
13953    [{'workTitle': 'QUIET CITY', 'conductorName': ...
Name: works, Length: 13954, dtype: object

We see (at least) two nested columns, concerts and works. Json_normalize docs give us some hints how to flatten semi-structured data further. Let's unpack the works column into a standalone dataframe.

In [25]:
works_data = pd.json_normalize(data=d['programs'], record_path='works', 
                            meta=['id', 'orchestra','programID', 'season', 'concerts'])
works_data.head(3)

,workTitle,conductorName,ID,soloists,composerName,movement,interval,movement.em,movement._,workTitle.em,workTitle._,id,orchestra,programID,season,concerts
0,"SYMPHONY NO. 5 IN C MINOR, OP.67","Hill, Ureli Corelli",52446*,[],"Beethoven, Ludwig van",NaN,NaN,NaN,NaN,NaN,NaN,38e072a7-8fc9-4f9a-8eac-3957905c0002,New York Philharmonic,3853,1842-43,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':..."
1,OBERON,"Timm, Henry C.",8834*4,"[{'soloistName': 'Otto, Antoinette', 'soloistR...","Weber, Carl Maria Von","""Ozean, du Ungeheuer"" (Ocean, thou mighty mons...",NaN,NaN,NaN,NaN,NaN,38e072a7-8fc9-4f9a-8eac-3957905c0002,New York Philharmonic,3853,1842-43,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':..."
2,"QUINTET, PIANO, D MINOR, OP. 74",NaN,3642*,"[{'soloistName': 'Scharfenberg, William', 'sol...","Hummel, Johann",NaN,NaN,NaN,NaN,NaN,NaN,38e072a7-8fc9-4f9a-8eac-3957905c0002,New York Philharmonic,3853,1842-43,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':..."


In [26]:
works_data.columns

Index(['workTitle', 'conductorName', 'ID', 'soloists', 'composerName',
       'movement', 'interval', 'movement.em', 'movement._', 'workTitle.em',
       'workTitle._', 'id', 'orchestra', 'programID', 'season', 'concerts'],
      dtype='object')

In [ ]:
'id' 'orchestra'  'programID',  'season', 'concerts', works (now sflattened)

In [37]:
concerts_data = pd.json_normalize(data=d['programs'], record_path='concerts', 
                            meta=['id', 'orchestra','programID', 'season'
                                ])
concerts_data.head(3)

,Date,eventType,Venue,Location,Time,id,orchestra,programID,season
0,1842-12-07T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,38e072a7-8fc9-4f9a-8eac-3957905c0002,New York Philharmonic,3853,1842-43
1,1843-02-18T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,c7b2b95c-5e0b-431c-a340-5b37fc860b34,New York Philharmonic,5178,1842-43
2,1843-04-07T05:00:00Z,Special,Apollo Rooms,"Manhattan, NY",8:00PM,894e1a52-1ae5-4fa7-aec0-b99997555a37,Musicians from the New York Philharmonic,10785,1842-43


In [50]:
#Join concerts and works
merged = pd.merge(works_data, concerts_data, on = ['id', 'programID', 'season'])
merged.head()

,workTitle,conductorName,ID,soloists,composerName,movement,interval,movement.em,movement._,workTitle.em,...,orchestra_x,programID,season,concerts,Date,eventType,Venue,Location,Time,orchestra_y
0,"SYMPHONY NO. 5 IN C MINOR, OP.67","Hill, Ureli Corelli",52446*,[],"Beethoven, Ludwig van",NaN,NaN,NaN,NaN,NaN,...,New York Philharmonic,3853,1842-43,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':...",1842-12-07T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,New York Philharmonic
1,OBERON,"Timm, Henry C.",8834*4,"[{'soloistName': 'Otto, Antoinette', 'soloistR...","Weber, Carl Maria Von","""Ozean, du Ungeheuer"" (Ocean, thou mighty mons...",NaN,NaN,NaN,NaN,...,New York Philharmonic,3853,1842-43,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':...",1842-12-07T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,New York Philharmonic
2,"QUINTET, PIANO, D MINOR, OP. 74",NaN,3642*,"[{'soloistName': 'Scharfenberg, William', 'sol...","Hummel, Johann",NaN,NaN,NaN,NaN,NaN,...,New York Philharmonic,3853,1842-43,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':...",1842-12-07T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,New York Philharmonic
3,NaN,NaN,0*,[],NaN,NaN,Intermission,NaN,NaN,NaN,...,New York Philharmonic,3853,1842-43,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':...",1842-12-07T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,New York Philharmonic
4,OBERON,"Etienne, Denis G.",8834*3,[],"Weber, Carl Maria Von",Overture,NaN,NaN,NaN,NaN,...,New York Philharmonic,3853,1842-43,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':...",1842-12-07T05:00:00Z,Subscription Season,Apollo Rooms,"Manhattan, NY",8:00PM,New York Philharmonic


In [42]:
works_data.shape

(82793, 16)

In [43]:
concerts_data.shape

(21607, 9)

In [44]:
merged.shape

(120286, 22)

In [45]:
82793 + 21607

104400

Let's flatten the 'soloists' data here by passing a list. Since soloists is nested in works, we can pass that as:

In [32]:
soloist_data = pd.json_normalize(data=d['programs'], record_path=['works', 'soloists'], 
                            meta=['id', 'orchestra','programID', 'season', 'concerts'])
soloist_data.head(3)

,soloistName,soloistRoles,soloistInstrument,id,orchestra,programID,season,concerts
0,"Otto, Antoinette",S,Soprano,38e072a7-8fc9-4f9a-8eac-3957905c0002,New York Philharmonic,3853,1842-43,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':..."
1,"Scharfenberg, William",A,Piano,38e072a7-8fc9-4f9a-8eac-3957905c0002,New York Philharmonic,3853,1842-43,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':..."
2,"Hill, Ureli Corelli",A,Violin,38e072a7-8fc9-4f9a-8eac-3957905c0002,New York Philharmonic,3853,1842-43,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':..."


In [33]:
nycphil.head(2)

,season,orchestra,concerts,programID,works,id
0,1842-43,New York Philharmonic,"[{'Date': '1842-12-07T05:00:00Z', 'eventType':...",3853,"[{'workTitle': 'SYMPHONY NO. 5 IN C MINOR, OP....",38e072a7-8fc9-4f9a-8eac-3957905c0002
1,1842-43,New York Philharmonic,"[{'Date': '1843-02-18T05:00:00Z', 'eventType':...",5178,[{'workTitle': 'SYMPHONY NO. 3 IN E FLAT MAJOR...,c7b2b95c-5e0b-431c-a340-5b37fc860b34


In [35]:
unnest_data = pd.json_normalize(data=d['programs'], record_path=['concerts', 'works'], 
                            meta=['season', 'orchestra', 'programID', 'id'])
unnest_data.head(3)

KeyError: "Key 'works' not found. If specifying a record_path, all elements of data should have the path."